In [1]:
import re
import tweepy
import numpy as np
from tweepy import OAuthHandler
import os

In [2]:
%load_ext dotenv
%dotenv ./auth.txt

In [3]:
class TwitterClient(object):
    def __init__(self):
        try:
            self.auth = OAuthHandler(os.getenv('consumer_key'), os.getenv('consumer_secret'))
            self.auth.set_access_token(os.getenv('access_token'), os.getenv('access_token_secret'))
            self.api = tweepy.API(self.auth)
        except Tweepy.TweepError:
            print("Error: Authentication Failed")

    def get_tweets(self, query, count = 10):
        tweets = []
        try:
            fetched_tweets = self.api.search_tweets(q = query, count = count)
            for tweet in fetched_tweets:
                if tweet.text not in tweets:
                    tweets.append(tweet.text)
            return tweets
        except tweepy.TweepError as e:
            print("Error : " + str(e))
            
    def get_trending_tags(self):
        trends1 = self.api.get_place_trends(1)
        data = trends1[0] 
        trends = data['trends']
        names = []
        for trend in trends:
            if trend['name'][0]=='#':
                names.append(trend['name'])
        return names

In [4]:
api = TwitterClient()
# api.get_tweets(query = '#chess', count = 200)
api.get_trending_tags()

['#UFC269',
 '#RedBullBatalla',
 '#それスノ',
 '#FinalBattle',
 '#HBDSuperstarRajinikanth']

In [5]:
with open('data.txt', 'w') as f:
    tweets = api.get_tweets('#Rhea', 20000)
    for tweet in tweets:
        f.write(tweet + "_$_")

# Read Data

In [6]:
f = open('data.txt', 'r')
tweets = f.read().split('_$_')
f.close()

# Preprocess

In [7]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk import download
# download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    tweets_eng = []
    for word in tweets_clean:
        flag = True
        for i in word:
            if ord(i) >= 256:
                flag = False
                break
        if flag:
            tweets_eng.append(word)
    return tweets_eng

In [8]:
processed_tweets = [process_tweet(tweet) for tweet in tweets]
while [] in processed_tweets:
    processed_tweets.remove([])

In [9]:
len(processed_tweets)

66

In [10]:
def count_words(tweet:list, freqs:dict, wordToTweet:dict):
    for word in tweet:
        if word in freqs:
            freqs[word] += 1
            wordToTweet[word].append(tweet)
        else:
            freqs[word] = 1
            wordToTweet[word] = [tweet]
    return freqs

In [11]:
freqs = dict()
wordToTweet = dict()
for tweet in processed_tweets:
    count_words(tweet, freqs, wordToTweet)

In [12]:
freqs

{'holdssrculpritsaccount': 1,
 'kill': 1,
 'sushantsinghrajput': 4,
 'role': 1,
 'rhea': 50,
 'salmankhan': 1,
 'surajpancholi': 1,
 'via': 1,
 'telepath': 1,
 'link': 1,
 'zhiao': 2,
 'know': 3,
 'think': 1,
 'hehe': 1,
 'queso': 1,
 'cummi': 1,
 '...': 9,
 'nooo': 1,
 'beagl': 1,
 'run': 1,
 'part': 1,
 '4': 2,
 'seteth': 5,
 'neat': 1,
 'byleth': 2,
 'setleth': 1,
 'rhealeth': 1,
 'fe3h': 16,
 'fireemblemthreehous': 7,
 'eunjeong': 1,
 'heymama': 1,
 'cover': 1,
 'children': 1,
 'goddess': 1,
 'go': 3,
 'trick': 1,
 'treat': 1,
 'fehero': 3,
 'fireemblem': 9,
 'forgot': 1,
 'post': 1,
 'sothi': 3,
 'regalia': 1,
 'wish': 1,
 'church': 2,
 'peopl': 1,
 'react': 1,
 'fireemblemthre': 1,
 'one': 4,
 'titan': 4,
 'daughter': 2,
 'mytholog': 2,
 'greekmytholog': 2,
 'halloween': 2,
 'feh': 4,
 'witch': 1,
 'kumkum': 4,
 'bhagya': 4,
 'spoiler': 2,
 'alert': 1,
 'prachi': 4,
 'vow': 1,
 'reveng': 1,
 'follow': 4,
 'us': 4,
 'kumkumbhagya': 3,
 'yoi': 1,
 '..': 4,
 'henley': 1,
 'wrenley':

In [13]:
freq_sorted = list(freqs.items())
freq_sorted.sort(key = lambda x : -x[1])
freq_sorted[:10]

[('rhea', 50),
 ('fe3h', 16),
 ('...', 9),
 ('fireemblem', 9),
 ('ke', 8),
 ('fireemblemthreehous', 7),
 ('seteth', 5),
 ('sushantsinghrajput', 4),
 ('one', 4),
 ('titan', 4)]

In [14]:
def vectorize(tweet, alphabet):
    v = np.zeros(len(alphabet))
    for i in range(len(alphabet)):
        if alphabet[i] in tweet:
            v[i] += 1
    return v

In [15]:
def closestCluster(vector, centroids):
    closest = -1
    minDist = 2**30
    for key in centroids:
        dist = np.linalg.norm(centroids[key] - vector)
        if dist < minDist:
            minDist = dist
            closest = key
    return closest

In [16]:
def assignToCluster(clusters, vectors, centroids):
    for i in range(len(vectors)):
        c = closestCluster(vectors[i], centroids)
        clusters[c].append(i)
    return clusters

In [17]:
def kmeans(k, max_iter, vectors):
    clusters = {}
    centroids = {}
    idx = np.random.choice(len(vectors), k, replace=False)
    for i in range(k):
        clusters[i] = []
        centroids[i] = vectors[idx[i]] 
    clusters = assignToCluster(clusters, vectors, centroids)
    for _ in range(max_iter-1):
        for i in range(k):
            for j in clusters[i]:
                centroids[i] = centroids[i] + vectors[j]
            if clusters[i] != []:
                centroids[i] = centroids[i] / len(clusters[i])
            if len(clusters[i]):
                clusters[i].clear()
        clusters = assignToCluster(clusters, vectors, centroids)
    return clusters

In [18]:
alphabet = [i[0] for i in freq_sorted]
clusters = dict()
vectors = dict()
for i in range(len(processed_tweets)):
    vectors[i] = vectorize(processed_tweets[i], alphabet)
clusters[0] = [i for i in range(len(processed_tweets))]

In [19]:
clusters = kmeans(20, 100, vectors)

In [20]:
clusters

{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  8,
  9,
  10,
  11,
  13,
  14,
  18,
  19,
  20,
  21,
  22,
  23,
  25,
  27,
  31,
  32,
  34,
  37,
  39,
  40,
  48,
  50,
  54,
  56,
  57,
  58,
  59,
  60,
  61,
  63,
  64],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [62, 65],
 8: [16, 33, 35, 38],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [7,
  12,
  15,
  17,
  24,
  26,
  28,
  29,
  30,
  36,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  49,
  51,
  52,
  53,
  55],
 18: [],
 19: []}

In [23]:
print(len(alphabet))

384
